In [ ]:
%matplotlib inline

import glob
import matplotlib.pylab as plt
import numpy as np
import os
import struct
import warnings

from __future__ import print_function
from matplotlib.mlab import detrend
from obspy import *
from obspy.core import read, UTCDateTime
from obspy.clients.fdsn import Client
from obspy.signal.cross_correlation import xcorr_pick_correction
from obspy.signal.filter import lowpass
from obspy.signal.invsim import cosine_taper 
from scipy import interpolate, signal
from time import *

from lib_seis import wf_fft

warnings.filterwarnings('ignore')                  # do not show warnings

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['lines.linewidth'] = 0.5

# Adaptation of CAN_P.m by Carl Tape
# Coding by: Yongki Aiman, Tomy Gunawan, Angel Ling
# Analyzing the Sumatra P wave
# custom functions: wf_fft

In [ ]:
# get waveforms (all three components)
client = Client("IRIS")
t = UTCDateTime("2004-12-26T00:52:23.900")
starttime = t-(1*3600)    # 1 hr before the earthquake
endtime   = t+(3*3600)    # 3 hrs after the earthquake

st = client.get_waveforms("G", "CAN", "*", "B*", starttime, endtime, attach_response=True)
print(st)

In [ ]:
# plot all three components
can = st.copy()
can_e = can[0].data
can_n = can[1].data
can_z = can[2].data
times = can[0].times()

# FIX: do not hardcode station/channel
title_e = 'CAN (BHE) − starting '+ str(can[0].stats['starttime'])
title_n = 'CAN (BHN) − starting '+ str(can[0].stats['starttime'])
title_z = 'CAN (BHZ) − starting '+ str(can[0].stats['starttime'])

axx = [4500,5500]
fig, (ax1,ax2,ax3) = plt.subplots(3,1)
fig.set_size_inches(10,16)

ax1.plot(times,can_e, color='b')
ax1.set(xlabel='seconds',ylabel='counts',title=title_e)
#ax1.set_xlim(axx)

ax2.plot(times,can_n, color='b')
ax2.set(xlabel='seconds',ylabel='counts',title=title_n)
#ax2.set_xlim(axx)

ax3.plot(times,can_z, color='b')
ax3.set(xlabel='seconds',ylabel='counts',title=title_z)
#ax3.set_xlim(axx)

plt.show();

In [ ]:
# Z component only
plt.plot(times,can_z,color='b')
plt.title(title_z)
plt.xlabel('seconds')
plt.ylabel('counts')
plt.show();

In [ ]:
# make a copy
st1 = st.copy()

# Extract a subset time series containing the P wave
t1 = starttime+4000
t2 = starttime+6000

st1.trim(t1,t2)
print(st1)

# prepare to plot
y     = st1[0].data
times = st1[0].times()
npts  = st1[0].stats.npts
df    = st1[0].stats.sampling_rate
fNy   = df / 2.0

# plot
# FIX: do not hardcode station/channel
title = 'CAN (BHZ) − starting '+ str(st1[0].stats['starttime'])
plt.plot(times,st1[0].data, color='b')
plt.xlabel('seconds')
plt.ylabel('counts')
plt.title(title)
plt.show()

In [ ]:
# Do spectral analysis here

